In [17]:
import numpy as np
import pandas as pd
import os


In [18]:
file = open('../input/next-word/next_word.txt', encoding='utf8')
lines=[]

for i in file:
    lines.append(i)
lines[:5]

['\n',
 'One morning, when Gregor Samsa woke from troubled dreams, he found\n',
 'himself transformed in his bed into a horrible vermin.  He lay on\n',
 'his armour-like back, and if he lifted his head a little he could\n',
 'see his brown belly, slightly domed and divided by arches into stiff\n']

# Data Preprocessing

In [ ]:
data=''

for i in lines:
    data=' '.join(lines)
    
data = data.replace('\n','').replace('\r','')

In [20]:
data[:300]

' One morning, when Gregor Samsa woke from troubled dreams, he found himself transformed in his bed into a horrible vermin.  He lay on his armour-like back, and if he lifted his head a little he could see his brown belly, slightly domed and divided by arches into stiff sections.  The bedding was hard'

In [21]:
#Replace Punctuations with space
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

for i in data:
    if i in punc:
        data = data.replace(i,' ')
data[:300]

' One morning  when Gregor Samsa woke from troubled dreams  he found himself transformed in his bed into a horrible vermin   He lay on his armour like back  and if he lifted his head a little he could see his brown belly  slightly domed and divided by arches into stiff sections   The bedding was hard'

In [22]:
final = data.split()

In [23]:
final[:5]

['One', 'morning', 'when', 'Gregor', 'Samsa']


# Tokenization

In [24]:
from tensorflow.keras.preprocessing.text import Tokenizer


In [25]:
token = Tokenizer()
token.fit_on_texts(final)

seq_data = token.texts_to_sequences([final])

In [26]:
vc = len(token.word_index)+1
vc

3003

In [27]:
seq = []

for i in range(vc):
    words = seq_data[0][i-1:i+1]
    seq.append(words)
seq.remove(seq[0])
seq[:10]

[[59, 158],
 [158, 65],
 [65, 13],
 [13, 106],
 [106, 1082],
 [1082, 30],
 [30, 1537],
 [1537, 1083],
 [1083, 4],
 [4, 226]]

In [40]:
X=[]
y=[]

for i in seq:
    X.append(i[0])
    y.append(i[1])
X[:10],y[:10]

([59, 158, 65, 13, 106, 1082, 30, 1537, 1083, 4],
 [158, 65, 13, 106, 1082, 30, 1537, 1083, 4, 226])

In [52]:
from tensorflow.keras.utils import to_categorical
y=to_categorical(y,num_classes=vc)
y


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [53]:
X = np.array(X)
X,y

(array([ 59, 158,  65, ...,  70, 113, 326]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32))

# Building Model

In [31]:
from tensorflow.keras.layers import Embedding , LSTM , Dense
from tensorflow.keras.models import Sequential

In [34]:
model = Sequential()
model.add(Embedding(vc, 100, input_length=1))
model.add(LSTM(1000,return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation="relu"))
model.add(Dense(vc, activation="softmax"))

In [48]:
from tensorflow.keras.optimizers import Adam

In [49]:
model.compile(loss="categorical_crossentropy", optimizer=Adam(lr=0.001))

In [54]:
model.fit(X, y, epochs=150, batch_size=64)

2021-09-18 17:48:51.085102: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-18 17:48:51.097873: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2199995000 Hz


Epoch 1/150
47/47 [==============================] - 11s 148ms/step - loss: 7.5674
Epoch 2/150
47/47 [==============================] - 7s 152ms/step - loss: 5.7164
Epoch 3/150
47/47 [==============================] - 7s 147ms/step - loss: 5.6041
Epoch 4/150
47/47 [==============================] - 7s 149ms/step - loss: 5.4773
Epoch 5/150
47/47 [==============================] - 7s 145ms/step - loss: 5.4133
Epoch 6/150
47/47 [==============================] - 7s 147ms/step - loss: 5.4923
Epoch 7/150
47/47 [==============================] - 7s 150ms/step - loss: 5.4356
Epoch 8/150
47/47 [==============================] - 7s 145ms/step - loss: 5.4303
Epoch 9/150
47/47 [==============================] - 7s 146ms/step - loss: 5.2775
Epoch 10/150
47/47 [==============================] - 7s 145ms/step - loss: 5.0568
Epoch 11/150
47/47 [==============================] - 7s 146ms/step - loss: 4.9364
Epoch 12/150
47/47 [==============================] - 7s 151ms/step - loss: 4.7483
Epoch 13/150

In [137]:

def Predict_Next_Words(model, tokenizer, text):
    """
        In this function we are using the tokenizer and models trained
        and we are creating the sequence of the text entered and then
        using our model to predict and return the the predicted word.
    
    """
    for i in range(3):
        sequence = tokenizer.texts_to_sequences([text])[0]
        sequence = np.array(sequence)
        
        preds = model.predict_classes(sequence)
#         print(preds)
        predicted_word = ""
        
        for key, value in tokenizer.word_index.items():
            if value == preds:
                predicted_word = key
                break
        
        
        return predicted_word

In [142]:
text = input('Enter Your Text:- ')
text = text.split(" ")
text=text[-1]
text= ''.join(text)
Predict_Next_Words(model,token,text)

Enter Your Text:-  what a strenuous


'career'

In [143]:
def preprocess(text):
    text = text.split(" ")
    text=text[-1]
    text= ''.join(text)
    return text
    

In [145]:
text=input('Enter text : - ')
print('Predicted next : - ',Predict_Next_Words(model,token,preprocess(text)))

Enter text : -  hii sir good


Predicted next : -  morning


In [146]:
text=input('Enter text : - ')
print('Predicted next : - ',Predict_Next_Words(model,token,preprocess(text)))

Enter text : -  Getting up


Predicted next : -  at


In [169]:
text=input('Enter text : - ')
print('Predicted next : - ',Predict_Next_Words(model,token,preprocess(text)))

Enter text : -  i know that


Predicted next : -  he
